In [1]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successf

In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
def clone_repo(repo_url):
  repo_name = repo_url.split('/')[-1]
  local_path = f"/content/{repo_name}"
  Repo.clone_from(repo_url, str(local_path))
  return str(repo_name)

In [22]:
github_path = "https://github.com/CoderAgent/SecureAgent"

In [ ]:
path = clone_repo(github_path)

In [7]:
SUPPORTED_EXTENSIONS = [".py",".js",".tsx",".cpp",".ts",".java",".c"]
IGNORED_DIRS =["node_modules",".git","dist","__pycache__",".next","env",".vscode","venv"]

In [6]:
def get_file_content(file_path,repo_path):
  try:
    with open(file_path, 'r',encoding="utf-8") as file:
      content = file.read()

      rel_path = os.path.relpath(file_path,repo_path)

      return {
          "name": rel_path,
          "content": content
      }

  except Exception as e:
    print(f"Error reading file {file_path}: {e}")
    return None

In [9]:
def get_main_files_content(repo_path: str):
    """
    Get content of supported code files from the local repository.

    Args:
        repo_path: Path to the local repository

    Returns:
        List of dictionaries containing file names and contents
    """
    files_content = []

    try:
        for root, _, files in os.walk(repo_path):
            # Skip if current directory is in ignored directories
            if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
                continue

            # Process each file in current directory
            for file in files:
                file_path = os.path.join(root, file)
                if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                    file_content = get_file_content(file_path, repo_path)
                    if file_content:
                        files_content.append(file_content)

    except Exception as e:
        print(f"Error reading repository: {str(e)}")

    return files_content

In [10]:
file_content = get_main_files_content(path)

In [11]:
file_content

[{'name': 'src/prompts.ts',
  'content': 'import { encode, encodeChat } from "gpt-tokenizer";\nimport type { ChatCompletionMessageParam } from "groq-sdk/resources/chat/completions";\nimport type { PRFile } from "./constants";\nimport {\n  rawPatchStrategy,\n  smarterContextPatchStrategy,\n} from "./context/review";\nimport { GROQ_MODEL, type GroqChatModel } from "./llms/groq";\n\nconst ModelsToTokenLimits: Record<GroqChatModel, number> = {\n  "mixtral-8x7b-32768": 32768,\n  "gemma-7b-it": 32768,\n  "llama3-70b-8192": 8192,\n  "llama3-8b-8192": 8192,\n};\n\nexport const REVIEW_DIFF_PROMPT = `You are PR-Reviewer, a language model designed to review git pull requests.\nYour task is to provide constructive and concise feedback for the PR, and also provide meaningful code suggestions.\n\nExample PR Diff input:\n\'\n## src/file1.py\n\n@@ -12,5 +12,5 @@ def func1():\ncode line that already existed in the file...\ncode line that already existed in the file....\n-code line that was removed in t

In [13]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
  model = SentenceTransformer(model_name)
  return model.encode(text)

In [14]:
text = "I am a programmer"

embeddings = get_huggingface_embeddings(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
embeddings

array([ 1.81737654e-02, -3.02657508e-03, -4.77465875e-02,  1.86379403e-02,
        3.14537995e-02,  1.87255293e-02, -1.52534274e-02, -6.77293688e-02,
       -1.26903653e-02,  1.28427576e-02,  5.80701306e-02,  4.00234833e-02,
        3.27073298e-02,  7.12998435e-02,  5.56373373e-02,  1.68628506e-02,
        6.97603747e-02, -5.02619930e-02,  6.13140827e-03, -1.46559235e-02,
       -4.51957993e-03,  4.82934639e-02, -2.53051296e-02, -1.97862904e-03,
       -4.36902530e-02, -2.41507161e-02,  1.29505759e-02, -3.78611824e-03,
       -2.05718316e-02,  1.09819308e-01,  3.07672890e-03, -2.80443169e-02,
       -1.55807249e-02, -1.24789868e-02,  1.75239131e-06, -2.93756695e-03,
       -1.43048428e-02,  4.88386713e-02, -6.21114224e-02,  2.95061413e-02,
       -1.40470508e-02,  2.20708270e-02,  1.13067888e-02,  4.70893271e-02,
        7.58305984e-03, -8.30314530e-05,  6.67821169e-02, -1.21320095e-02,
        4.39386303e-03,  2.47453637e-02,  1.02529004e-02, -6.54432410e-03,
       -5.53147821e-03, -

In [18]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

pinecone_index = pc.Index("codebase-rag")

In [19]:
vectorstore = PineconeVectorStore(index_name="codebase-rag",embedding=HuggingFaceEmbeddings())

<ipython-input-19-a7c8cea40874>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = PineconeVectorStore(index_name="codebase-rag",embedding=HuggingFaceEmbeddings())
<ipython-input-19-a7c8cea40874>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name="codebase-rag",embedding=HuggingFaceEmbeddings())


In [24]:
documents = []

for file in file_content:
  doc = Document(
      page_content =f"{file['name']}\n{file['content']}",
      metadata = {"source": file["name"]}
  )
  documents.append(doc)

vectorstore = PineconeVectorStore.from_documents(documents=documents,
                                                  embedding=HuggingFaceEmbeddings(),
                                                  index_name="codebase-rag",
                                                  namespace =github_path)

<ipython-input-24-8955ab4ddc4f>:11: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


In [47]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=userdata.get("GROQ_API_KEY")
)

In [26]:
query = "How is the  JavaScript parser used?"

In [27]:
query_embedding = get_huggingface_embeddings(query)

In [28]:
query_embedding

array([ 5.71991131e-02, -3.48081291e-02, -3.27215418e-02,  5.29010817e-02,
       -3.88054959e-02,  2.21033078e-02,  1.60938818e-02, -1.00735975e-02,
        3.04608904e-02, -6.25874922e-02,  2.71743089e-02,  3.67565900e-02,
        5.69119453e-02,  5.45354113e-02,  4.02487814e-02, -4.73744869e-02,
        3.59910820e-03,  6.65521948e-03,  1.47536704e-02,  3.57538760e-02,
        1.81228686e-02,  1.24748386e-02, -2.07926128e-02,  6.99328259e-02,
       -1.78905539e-02, -1.98267922e-02, -8.77424423e-03, -4.04769043e-03,
       -4.82026935e-02, -1.55368652e-02, -6.26485124e-02, -6.66373223e-03,
        1.43068153e-02, -4.92968187e-02,  1.30648721e-06, -2.02893210e-03,
       -4.47639599e-02,  2.07317546e-02, -2.80540297e-03,  1.37846796e-02,
        4.11506603e-03,  6.87661488e-03, -2.91272085e-02, -6.68382505e-03,
        2.94112526e-02, -4.13797721e-02,  3.90248671e-02, -5.73173016e-02,
        3.29415090e-02,  1.95522374e-03, -7.05714687e-04, -2.74959207e-02,
        8.47589783e-03,  

In [30]:
top_matches = pinecone_index.query(vector=query_embedding.tolist(), top_k=5, include_metadata=True, namespace=github_path)

In [31]:
top_matches

{'matches': [{'id': '6bf23446-b947-4d8f-8b7f-03fbb7217f10',
              'metadata': {'source': 'src/context/language/javascript-parser.ts',
                           'text': 'src/context/language/javascript-parser.ts\n'
                                   'import { AbstractParser, EnclosingContext '
                                   '} from "../../constants";\n'
                                   'import * as parser from "@babel/parser";\n'
                                   'import traverse, { NodePath, Node } from '
                                   '"@babel/traverse";\n'
                                   '\n'
                                   'const processNode = (\n'
                                   '  path: NodePath<Node>,\n'
                                   '  lineStart: number,\n'
                                   '  lineEnd: number,\n'
                                   '  largestSize: number,\n'
                                   '  largestEnclosingContext: Node | n

In [32]:
contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [33]:
contexts

['src/context/language/javascript-parser.ts\nimport { AbstractParser, EnclosingContext } from "../../constants";\nimport * as parser from "@babel/parser";\nimport traverse, { NodePath, Node } from "@babel/traverse";\n\nconst processNode = (\n  path: NodePath<Node>,\n  lineStart: number,\n  lineEnd: number,\n  largestSize: number,\n  largestEnclosingContext: Node | null\n) => {\n  const { start, end } = path.node.loc;\n  if (start.line <= lineStart && lineEnd <= end.line) {\n    const size = end.line - start.line;\n    if (size > largestSize) {\n      largestSize = size;\n      largestEnclosingContext = path.node;\n    }\n  }\n  return { largestSize, largestEnclosingContext };\n};\n\nexport class JavascriptParser implements AbstractParser {\n  findEnclosingContext(\n    file: string,\n    lineStart: number,\n    lineEnd: number\n  ): EnclosingContext {\n    const ast = parser.parse(file, {\n      sourceType: "module",\n      plugins: ["jsx", "typescript"], // To allow JSX and TypeScript

In [34]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [35]:
print(augmented_query)

<CONTEXT>
src/context/language/javascript-parser.ts
import { AbstractParser, EnclosingContext } from "../../constants";
import * as parser from "@babel/parser";
import traverse, { NodePath, Node } from "@babel/traverse";

const processNode = (
  path: NodePath<Node>,
  lineStart: number,
  lineEnd: number,
  largestSize: number,
  largestEnclosingContext: Node | null
) => {
  const { start, end } = path.node.loc;
  if (start.line <= lineStart && lineEnd <= end.line) {
    const size = end.line - start.line;
    if (size > largestSize) {
      largestSize = size;
      largestEnclosingContext = path.node;
    }
  }
  return { largestSize, largestEnclosingContext };
};

export class JavascriptParser implements AbstractParser {
  findEnclosingContext(
    file: string,
    lineStart: number,
    lineEnd: number
  ): EnclosingContext {
    const ast = parser.parse(file, {
      sourceType: "module",
      plugins: ["jsx", "typescript"], // To allow JSX and TypeScript
    });
    let larges

In [54]:
system_prompt = f"""You are a Principle Software Engineer, with over 20 years of expertise in TypeScript.

Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
"""

llm_response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content

In [55]:
print(response)

The JavaScript parser is used in the following ways:

1. **`JavascriptParser` class:** The `JavascriptParser` class is implemented in `src/context/language/javascript-parser.ts`. It contains two methods: `findEnclosingContext` and `dryRun`.

   a. `findEnclosingContext` method: This method is used to find the enclosing context of a given function in the provided JavaScript file. It uses the `parser.parse` function from `@babel/parser` to parse the file into an Abstract Syntax Tree (AST) and then iterates over the AST using the `babel/traverse` function to find the enclosing function.

   b. `dryRun` method: This method is used to perform a dry run of the parser. It attempts to parse the provided JavaScript file and returns a valid object if parsing is successful, or an error object if parsing fails.

2. **`getParserForExtension` function:** The `getParserForExtension` function in `src/constants.ts` is used to retrieve the parser for a given file extension. If the file extension is ".ts

In [56]:
def perform_rag(query):
    raw_query_embedding = get_huggingface_embeddings(query)

    top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/CoderAgent/SecureAgent")

    # Get the list of retrieved texts
    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

    # Modify the prompt below as need to improve the response quality
    system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.

    Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
    """

    llm_response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    return llm_response.choices[0].message.content

In [58]:
response = perform_rag("How is the python parser used?")

print(response)

The Python parser is used in the `src/context/review.ts` file. 

In the `smarterContextPatchStrategy` function, the filename's extension is checked to determine the type of parser to use. If the extension matches a supported programming language (not explicitly mentioned in this code snippet, but implied to be `.py` for Python), the Python parser is used to determine the context in which patches were made in the code.

The function `functionContextPatchStrategy` is then called with the Python parser, which attempts to find the enclosing context for the changes made in the file.

Here is the relevant code snippet:

```typescript
const parser: AbstractParser = getParserForExtension(file.filename);
if (parser != null) {
  return functionContextPatchStrategy(file, parser);
} else {
  return expandedPatchStrategy(file);
}
```

However, there seems to be a minor mishap in this part of the code, as the `PythonParser` is never imported or instanced, which would mean the Python parser code in `

In [61]:
!pip install pyngrok streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.3 MB/s eta 0:00:00


In [72]:
from threading import Thread
from pyngrok import ngrok
from google.colab import userdata
ngrok_token = userdata.get('NGROK_AUTH_TOKEN')

ngrok.set_auth_token(ngrok_token)

In [151]:
PORT = 8530
app_path = "/content/app.py"

def run_streamlit():
  os.system(f"streamlit run {app_path} --server.port {PORT}")

In [152]:
%%writefile .env

PINECONE_API_KEY= pcsk_2ccXoj_R7MGS2BfDnev7QcBWYZNdSwWSAdeoKzpTwrGE5eW5ZjmeoVLCSfgiq4hAqZP8WF
GROQ_API_KEY =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

Overwriting .env


In [138]:
!pip install python-dotenv

In [153]:
%%writefile app.py
import os
import shutil
import streamlit as st
from sentence_transformers import SentenceTransformer
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from git import Repo
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Pinecone
def init_pinecone():
    api_key = os.getenv("PINECONE_API_KEY")
    if not api_key:
        raise ValueError("Pinecone API key not set!")

    pc = Pinecone(api_key=api_key)
    index_name = "codebase-rag"

    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=768,
            metric="cosine",
            spec=ServerlessSpec(
                cloud="aws",
                region="us-west-1"
            )
        )

    index = pc.Index(index_name)
    if index is None:
        raise ValueError("Failed to initialize Pinecone index.")
    return pc, index

# Clone a GitHub repository
@st.cache_data(show_spinner=False)
def clone_repo(repo_url):
    repo_name = repo_url.split("/")[-1]
    local_path = f"/tmp/{repo_name}"
    if os.path.exists(local_path):
        shutil.rmtree(local_path)
    Repo.clone_from(repo_url, local_path)
    return local_path

# Get content of files in the repository
def get_main_files_content(repo_path):
    SUPPORTED_EXTENSIONS = {".py", ".ts", ".js", ".jsx", ".tsx", ".json", ".html", ".css"}
    IGNORED_DIRS = {".git", "__pycache__", "node_modules"}
    files_content = []

    for root, _, files in os.walk(repo_path):
        if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
            continue

        for file in files:
            if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding="utf-8") as f:
                    rel_path = os.path.relpath(file_path, repo_path)
                    content = f.read()
                    files_content.append({"name": rel_path, "content": content})
    return files_content

# Create a Pinecone vectorstore
@st.cache_resource(show_spinner=False)
def create_vectorstore(repo_path):
    embedding_model = HuggingFaceEmbeddings()
    index_name = "codebase-rag"
    _, pinecone_index = init_pinecone()

    files_content = get_main_files_content(repo_path)
    if not files_content:
        raise ValueError("No files found in the repository for indexing.")

    documents = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=200)
    for file in files_content:
        chunks = text_splitter.split_text(file["content"])
        for i, chunk in enumerate(chunks):
            doc = Document(
                page_content=chunk,
                metadata={"source": f"{file['name']} - chunk {i}"}
            )
            documents.append(doc)

    vectorstore = PineconeVectorStore.from_documents(
        documents=documents,
        embedding=embedding_model,
        index_name=index_name
    )

    if vectorstore is None:
        raise ValueError("Failed to create vectorstore.")
    return vectorstore

# Get embeddings for a query
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

# Perform RAG (Retrieval Augmented Generation)
def perform_rag(query, vectorstore):
    pinecone_index = vectorstore._index
    raw_query_embedding = get_huggingface_embeddings(query)
    top_matches = pinecone_index.query(
        vector=raw_query_embedding.tolist(),
        top_k=5,
        include_metadata=True
    )

    if not top_matches["matches"]:
        return "No relevant documents found."

    contexts = [item["metadata"]["source"] for item in top_matches["matches"]]
    augmented_query = f"<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[:10]) + "\n-------\n</CONTEXT>\nMY QUESTION:\n" + query
    system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.

    Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
    """
    client = OpenAI(
        base_url="https://api.groq.com/openai/v1",
        api_key=os.getenv("GROQ_API_KEY")
    )

    llm_response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    return llm_response.choices[0].message.content

# Streamlit app
st.title("Chat with CodeBot")

if "messages" not in st.session_state:
    st.session_state.messages = [{"role": "system", "content": "You are a helpful assistant."}]

repo_url = st.text_input("Enter GitHub repository URL:")

if repo_url and "repo_path" not in st.session_state:
    if st.button("Clone and Index Repository"):
        with st.spinner("Cloning the repository and indexing..."):
            try:
                repo_path = clone_repo(repo_url)
                vectorstore = create_vectorstore(repo_path)
                st.session_state["repo_path"] = repo_path
                st.session_state["vectorstore"] = vectorstore
                st.success("Repository indexed successfully!")
            except Exception as e:
                st.error(f"Error: {e}")

if "vectorstore" not in st.session_state:
    st.warning("Please index a repository before asking questions.")
else:
    user_input = st.text_input("Your question:", key="user_input", on_change=None)

    if user_input:
        st.session_state.messages.append({"role": "user", "content": user_input})

        with st.spinner("Thinking..."):
            try:
                response = perform_rag(user_input, st.session_state["vectorstore"])
            except Exception as e:
                response = f"Error: {e}"

        st.session_state.messages.append({"role": "assistant", "content": response})
        for msg in st.session_state.messages:
            if msg["role"] == "user":
                st.markdown(f"**You:** {msg['content']}")
            elif msg["role"] == "assistant":
                st.markdown(f"**CodeBot:** {msg['content']}")

Overwriting app.py


In [155]:
thread = Thread(target=run_streamlit)
thread.start()

In [156]:
proto='http'
public_url = ngrok.connect(addr=str(PORT),proto=proto,bind_tls=True)

print("Public URL:",public_url)

Public URL: NgrokTunnel: "https://4d9f-34-106-6-2.ngrok-free.app" -> "http://localhost:8530"


In [154]:
tunnels = ngrok.get_tunnels()
for tunnel in tunnels:
  print(f"Closing tunnel: {tunnel.public_url} -> {tunnel.config['addr']}")
  ngrok.disconnect(tunnel.public_url)

Closing tunnel: https://0c2d-34-106-6-2.ngrok-free.app -> http://localhost:8529
Closing tunnel: https://1881-34-106-6-2.ngrok-free.app -> http://localhost:8527
Closing tunnel: https://fad6-34-106-6-2.ngrok-free.app -> http://localhost:8527
